# Chapter 14: Deploying TensorFlow Models to Production

## 1. Pendahuluan

Bab ini membahas bagaimana cara mendistribusikan dan mengimplementasikan model TensorFlow ke lingkungan produksi. Setelah model dilatih dan divalidasi, tantangan selanjutnya adalah bagaimana melayaninya (serving), menyimpan, mengelola versi, dan melakukan inferensi secara cepat dan andal.

Bagian ini juga mencakup cara menggunakan TensorFlow Serving, SavedModel, dan integrasi dengan REST API untuk memudahkan deployment ke aplikasi nyata.


## 2. Menyimpan Model dengan SavedModel

TensorFlow menyediakan format universal bernama `SavedModel` yang dapat menyimpan:

- Arsitektur model
- Bobot terlatih
- Informasi tentang signature (input dan output)
- Optimizer state (jika perlu melanjutkan training)

Contoh cara menyimpan model:

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")
# Misalnya sudah dilatih...

# Menyimpan model ke folder
model.save("my_model")


Untuk memuatnya kembali:

In [ ]:
loaded_model = tf.keras.models.load_model("my_model")


## 3. TensorFlow Serving

TensorFlow Serving adalah sistem fleksibel dan berperforma tinggi untuk menyajikan model machine learning di produksi. Keunggulannya:

- Mendukung hot-swapping model (versi lama ke versi baru tanpa downtime)
- Mendukung gRPC dan REST API
- Mendukung batching request secara otomatis

Langkah dasar untuk menggunakan TensorFlow Serving:

1. Simpan model menggunakan format SavedModel.
2. Jalankan TensorFlow Serving Docker container atau instal binary di server.
3. Konfigurasi path ke model.
4. Kirim request inferensi via REST API atau gRPC.

Contoh command Docker untuk menjalankan TensorFlow Serving:

In [ ]:
docker run -p 8501:8501 \
  --mount type=bind,source=$(pwd)/my_model,target=/models/my_model \
  -e MODEL_NAME=my_model -t tensorflow/serving


Model akan tersedia di:

* REST API: http://localhost:8501/v1/models/my_model:predict
* gRPC: port 8500



## 4. Batching dan Autotuning

TensorFlow Serving dapat melakukan batching request secara otomatis. Ini berguna untuk meningkatkan throughput server inferensi pada beban tinggi.

Fitur ini menggabungkan beberapa permintaan menjadi satu batch besar dan mengirimkannya ke model sekaligus. Dengan demikian, inferensi bisa lebih efisien terutama di GPU.


## 5. Signature dan API

Saat menyimpan model, TensorFlow menyimpan signature yang mendefinisikan nama input dan output. Signature mempermudah klien untuk tahu cara mengirim data ke model.

Contoh memeriksa signature model:


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model("my_model")
model.signatures


## 6. Membuat REST Client

Client inferensi bisa dibuat dalam berbagai bahasa. Berikut contoh sederhana menggunakan Python dan `requests` untuk mengirim JSON ke TensorFlow Serving:


In [ ]:
import requests
import json

data = json.dumps({"instances": [[1.0, 2.0, 5.0, 8.0]]})
response = requests.post("http://localhost:8501/v1/models/my_model:predict", data=data)
print(response.json())


## 7. Model Versioning

TensorFlow Serving mendukung multiple version untuk model yang sama. Versi model diletakkan dalam subdirektori:




In [ ]:
/models/my_model/1/
/models/my_model/2/

TensorFlow Serving otomatis akan menggunakan versi terbaru secara default, tetapi klien juga bisa meminta versi tertentu.

## 8. Monitoring dan Logging

Dalam produksi, penting untuk memonitor performa model, latency, throughput, dan error rate. Anda bisa menggunakan tools seperti Prometheus, Grafana, atau Cloud Monitoring. Logging input dan output juga membantu untuk audit dan debugging.


## 9. Alternatif Deployment

Selain TensorFlow Serving, ada beberapa alternatif deployment:

- Gunakan model di dalam aplikasi Python dengan Flask atau FastAPI.
- Deploy ke cloud service seperti Google AI Platform, AWS SageMaker, atau Azure ML.
- Export ke format lain seperti TensorFlow Lite atau TensorFlow.js untuk aplikasi mobile dan web.
